In [1]:
!pip install -e "git+https://github.com/daffarobani/smt-explainability.git"#egg=smt-ex
!pip install sobolsampling

Obtaining smt-ex from git+https://github.com/daffarobani/smt-explainability.git#egg=smt-ex
  Updating ./src/smt-ex clone
  Running command git fetch -q --tags
  Running command git reset --hard -q f11aeac616575ebe61df3c87288c1b891c780ac7
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for smt-ex (pyproject.toml) ... done
  Created wheel for smt-ex: filename=SMT_EX-0.1.0-0.editable-py3-none-any.whl size=3879 sha256=da3b024ef5e8a1b9a7a94439f11b94563e595ebcf012057261effa8a4eb96c75
  Stored in directory: /tmp/pip-ephem-wheel-cache-4bc08idh/wheels/15/62/c0/cc1e0450c8bbc4336537f9dc178005e194cb783cd9399062e3
Successfully built smt-ex
  Attempting uninstall: smt-ex
    Found existing installation: SMT-EX 0.1.0
    Uninstalling SMT-EX-0.1.0:
      Successfully uninstalled SMT-EX-0.1.0
  Using cached sobolsampling-0.1.

In [2]:
import numpy as np
import math

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from smt.surrogate_models import KRG
from smt.problems import WingWeight
from smt.sampling_methods import LHS

from smt_ex.sobol import SobolIndices
from smt_ex.sobol import SobolIndicesDisplay

In [3]:
ndoe = 100
fun = WingWeight()
sampling = LHS(xlimits=fun.xlimits, criterion='ese', random_state=1)
X = sampling(ndoe)
y = fun(X)

n_train = int(0.8 * ndoe)

X_tr, y_tr = X[:n_train, :], y[:n_train]
X_te, y_te = X[n_train:, :], y[n_train:]

## config
feature_names = [
    r'$S_{w}$', r'$W_{fw}$', r'$A$', r'$\Delta$',
    r'$q$', r'$\lambda$', r'$t_{c}$', r'$N_{z}$',
    r'$W_{dg}$', r'$W_{p}$',
]

class GroundTruthModel:
    def predict_values(self, X):
        return fun(X)

# kriging model
sm = KRG(
    theta0=[1e-2]*X_tr.shape[1],
    print_prediction = False
    )
sm.set_training_values(
    X_tr,
    y_tr
    )

sm.train()

print("Surrogate model")
y_pred = sm.predict_values(X_te)
rmse = mean_squared_error(y_te, y_pred, squared=False)
rrmse = rmse / y_te.mean()
print(f"RMSE: {rmse:.4f}")
print(f"rRMSE: {rrmse:.4f}")

# ground truth model
gtm = GroundTruthModel()